# Tdem Data Point Class

## Tdem Data contains entire data sets
## Tdem Data Points can forward model and evaluate themselves

### TdemData is an extension to the [Data](Data.ipynb) Class

##### Back to [Main](../../PackageInfo.ipynb)

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

from os.path import join
import numpy as np
import h5py
import matplotlib.pyplot as plt
from geobipy import hdfRead
from geobipy import TdemData
from geobipy import TdemDataPoint
from geobipy import Model1D
from geobipy import StatArray

In [ ]:
# The data file name
dataFile=[join('supplementary','Data','Skytem_High.txt'), join('supplementary','Data','Skytem_Low.txt')]
# The EM system file name
systemFile=[join('supplementary','Data','SkytemHM-SLV.stm'), join('supplementary','Data','SkytemLM-SLV.stm')]

## Initialize and read an EM data set

In [ ]:
D=TdemData()
D.read(dataFile,systemFile)

## Summarize the Data

In [ ]:
D.summary()

## Grab a measurement from the data set

In [ ]:
P=D.getDataPoint(0)
P.s[:] = 1e-12
P.summary()
plt.figure()
P.plot()

## We can forward model the EM response of a 1D layered earth <a href="../Model/Model1D.ipynb">Model1D</a>

In [ ]:
par = StatArray(np.linspace(0.01, 0.1, 10), "Conductivity", "$\\frac{S}{m}$")
thk = StatArray(np.ones(10) * 10.0)
mod = Model1D(nCells = 10, parameters=par, thickness=thk)
mod.summary()
plt.figure()
mod.pcolor()

## Compute and plot the data from the model

In [ ]:
P.forward(mod)
plt.figure()
P.plot()
P.plotPredicted()

## The errors are set to zero right now, so lets change that

In [ ]:
# Set the Prior
P.p.setPrior('MVNormalLog' ,P.d[P.iActive], P.s[P.iActive]**2.0)
#P.updateErrors(option=2, err=P.s[:], relativeErr=[0.05,0.05], additiveErr=[1e-12,1e-13])

## With forward modelling, we can solve for the best fitting halfspace model

In [ ]:
HSconductivity=P.FindBestHalfSpace()
print(HSconductivity)
plt.figure()
P.plot(withErr=True)
P.plotPredicted()

## Compute the misfit between observed and predicted data

In [ ]:
print(P.dataMisfit())

## Plot the misfits for a range of half space conductivities

In [ ]:
plt.figure()
P.plotHalfSpaceResponses(-6.0,4.0,200)

## Compute the sensitivity matrix for a given model

In [ ]:
sensitivityMatrix = P.sensitivity(mod)
J=StatArray(sensitivityMatrix,'|Sensitivity|')
plt.figure()
np.abs(J).pcolor(grid=True, log=10, equalize=True, linewidth=1)

## We can save the FdemDataPoint to a HDF file

In [ ]:
with h5py.File('TdemDataPoint.h5','w') as hf:
    P.writeHdf(hf, 'tdp')

## And then read it in

In [ ]:
P1=hdfRead.readKeyFromFiles('TdemDataPoint.h5','/','tdp', sysPath=join('supplementary','Data'))

In [ ]:
P1.summary()